Study the relationship between the velocity width of HI detections and the distance to the nearest neighbor for MaNGA galaxies.

In [27]:
from astropy.table import Table
import numpy as np
from scipy.spatial import KDTree

# Data

In [28]:
mangaHI_filename = 'input/mangaHIall.fits'
alfaHI_filename = 'input/manga_dr15_alfalfa.fits'
manga_filename =  'input/Pipe3D-master_file_vflag_10_smooth2p27_N2O2_noWords.txt'
#kias7_filename =  'kias1033_5_P-MJD-F_MPAJHU_ZdustOS_stellarMass_BPT_SFR_NSA_vflag.txt'
kias7_filename =  'input/kias1033_5_P-MJD-F_MPAJHU_ZdustOS_stellarMass_BPT_SFR_NSA_vflag.txt'

mangaHI = Table.read(mangaHI_filename, format='fits')
alfaHI = Table.read(alfaHI_filename, format='fits')
manga = Table.read(manga_filename, format='ascii.commented_header')
kias7 = Table.read(kias7_filename, format='ascii.commented_header')

## Add HI data to MaNGA table

##### Initialize new columns in MaNGA data table

In [30]:
manga['WF50'] = np.NaN*np.ones(len(manga), dtype=float)
manga['WP20'] = np.NaN*np.ones(len(manga), dtype=float)
manga['logHI'] = np.NaN*np.ones(len(manga), dtype=float)

##### Add ALFALFA HI data

In [31]:
for i in range(len(alfaHI)):
    
    galID = alfaHI['PLATEIFU'][i]
    plate, IFU = galID.split('-')
    
    # Find the corresponding row index for this galaxy in the MaNGA data table
    plate_boolean = manga['MaNGA_plate'] == int(plate)
    IFU_boolean = manga['MaNGA_IFU'] == int(IFU)
    manga_boolean = np.logical_and(plate_boolean, IFU_boolean)
    
    # Add data to MaNGA data table
    manga['WF50'][manga_boolean] = alfaHI['WF50'][i]
    manga['WP20'][manga_boolean] = alfaHI['WP20'][i]
    manga['logHI'][manga_boolean] = alfaHI['LOGMHI'][i]

##### Add GBT HI data

In [32]:
for i in range(len(mangaHI)):
    
    galID = mangaHI['plateifu'][i]
    plate, IFU = galID.split('-')
    
    # Find the corresponding row index for this galaxy in the MaNGA data table
    plate_boolean = manga['MaNGA_plate'] == int(plate)
    IFU_boolean = manga['MaNGA_IFU'] == int(IFU)
    manga_boolean = np.logical_and(plate_boolean, IFU_boolean)
    
    # Add data to MaNGA data table
    manga['WF50'][manga_boolean] = mangaHI['WF50'][i]
    manga['WP20'][manga_boolean] = mangaHI['WP20'][i]
    manga['logHI'][manga_boolean] = mangaHI['logMHI'][i]

## Isolate those MaNGA galaxies with HI detections

We only want to analyze those galaxies in MaNGA with HI detections, so let's create a subset of the main MaNGA data table that contains only those galaxies with HI detections.

In [33]:
HI_boolean = np.logical_or.reduce((np.isfinite(manga['WF50']), 
                                   np.isfinite(manga['WP20']), 
                                   np.isfinite(manga['logHI'])))

manga_HI = manga[HI_boolean]

# Find distance to nearest neighbor for each MaNGA galaxy

For each galaxy in the MaNGA data table, find its nearest neighbor and calculate the distance to that galaxy.
1. Calculate the distance to every galaxy in SDSS DR7 for each MaNGA galaxy with an HI detection.
2. Find the galaxy with the smallest distance - this is our nearest neighbor!  (Note: We need to check to make sure that we do not identify the MaNGA HI target galaxy as the nearest neighbor.)

### Convert (ra, dec, redshift) to (x, y, z)

Since we are going to do this for both the KIAS galaxies and the MaNGA galaxies, let's write a function.

In [34]:
c = 3e5 #3*(10**8) m/s <-- Scientific notation
H = 100 #2.43 * (10**-18) 1/s <-- We are doing to do all of our distances in Mpc/h (where H0 = 100h).

def rdz_to_xyz(data):
    '''
    Convert ra, dec, redshift to x, y, z
    
    
    PARAMETERS
    ==========
    
    data : astropy table
        Galaxies to analyze.  Columns include ra, dec, redshift.
        
        
    RETURNS
    =======
    
    data : astropy table
        Same as input data object, with the additional fields of x, y, z
    '''

    '''
    # We can do this all at once, since we are doing the same calculation on every 
    # row of the data table.
    
    for i in range(len(data)):
        radius_2 = c*data['redshift']/H
        r_2 = radius_2
        x_2_cor = r_2 * np.cos(kias7['ra'][i]* np.pi/180) * np.cos(kias7['dec'][i]* np.pi/180)
        y_2_cor = r_2 * np.sin(kias7['ra'][i]* np.pi/180) * np.cos(kias7['dec'][i]* np.pi/180)
        z_2_cor = r_2 * np.sin(kias7['ra'][i]* np.pi/180)
        x_2.append(x_2_cor)
        y_2.append(y_2_cor)
        z_2.append(z_2_cor)
    '''
    
    ################################################################################
    # Our data tables have different column names, so we need to determine the 
    # correct names.
    #-------------------------------------------------------------------------------
    if 'redshift' in data.colnames:
        redshift = data['redshift']
        ra = data['ra'] * np.pi/180
        dec = data['dec'] * np.pi/180
    elif 'NSA_redshift' in data.colnames:
        redshift = data['NSA_redshift']
        ra = data['NSA_RA'] * np.pi/180
        dec = data['NSA_DEC'] * np.pi/180
    else:
        print('Unknown field names.  Add to rdz_to_xyz function and rerun.')
        return data
    ################################################################################
    
    
    ################################################################################
    # Convert from spherical to cartesian coordinates
    #-------------------------------------------------------------------------------
    r = c*redshift/H
    
    data['x'] = r * np.cos(ra) * np.cos(dec)
    data['y'] = r * np.sin(ra) * np.cos(dec)
    data['z'] = r * np.sin(dec)
    ################################################################################
    
    return data

In [42]:
# Now that we have our function written, we can do this conversion on both data tables
manga_HI = rdz_to_xyz(manga_HI)
kias7 = rdz_to_xyz(kias7)

### Calculate distance between galaxies and find the closest one

In [37]:
# Generates x_tree, y_tree, z_tree array for kias7
x_tree = np.asarray(kias7['x'])
x_tree = x_tree.astype(np.float64)
x_tree = x_tree.reshape(x_tree.shape[0],1)
y_tree = np.asarray(kias7['y'])
y_tree = y_tree.astype(np.float64)
y_tree = y_tree.reshape(y_tree.shape[0],1)
z_tree = np.asarray(kias7['z'])
z_tree = z_tree.astype(np.float64)
z_tree = z_tree.reshape(z_tree.shape[0],1)
xyz_tree = np.concatenate((x_tree,y_tree,z_tree), axis=1)

# Generates x_query, y_query, z_query array for manga
x_query = np.asarray(manga_HI['x'])
x_query = x_query.astype(np.float64)
x_query = x_query.reshape(x_query.shape[0],1)
y_query = np.asarray(manga_HI['y'])
y_query = y_query.astype(np.float64)
y_query = y_query.reshape(y_query.shape[0],1)
z_query = np.asarray(manga_HI['z'])
z_query = z_query.astype(np.float64)
z_query = z_query.reshape(z_query.shape[0],1)
xyz_query = np.concatenate((x_query,y_query,z_query), axis=1)

In [38]:
# Generate KDTree for kias7
tree = KDTree(xyz_tree)

In [39]:
nearest_distances, _ = tree.query(xyz_query, k=2)
nearest_distances = nearest_distances[:,1]
nearest_distances

array([1.23878972, 0.657584  , 0.35897515, ..., 0.2109972 , 0.47263304,
       0.87133691])

In [40]:
manga_HI['d_nn'] = np.zeros(len(manga_HI), dtype=float)
manga_HI['d_nn'] = nearest_distances